as shown negatively [here](./glm_R_vs_lassoglm_MATLAB_real_data.ipynb), changing nonlinearity doesn't seem to improve performance. maybe performance can be improved instead by using minibatch.

In [1]:
from sklearn.decomposition import PCA
import pickle
import os.path
from torch.backends import cudnn
cudnn.benchmark = False
cudnn.enabled = False

In [2]:
from scipy.stats import pearsonr
import numpy as np

In [3]:
# load data
from tang_jcompneuro.io import load_split_dataset
from tang_jcompneuro.configs.cnn_arch import generate_one_config
from tang_jcompneuro.configs.cnn_init import legacy_generator
from tang_jcompneuro.configs import cnn_opt
from tang_jcompneuro.cnn import CNN
from tang_jcompneuro.training_aux import train_one_case
from tang_jcompneuro import dir_dictionary
from collections import OrderedDict

In [4]:
def load_withoutval_set():
    datasets = load_split_dataset('MkA_Shape', 'all', False, 553)
    X_train, y_train, X_test, y_test = datasets[:4]
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)


    # random stuff is faster.
    pca_obj_this = PCA(svd_solver='randomized', random_state=0)
    pca_obj_this.fit(X_train)
    X_train_pca = pca_obj_this.transform(X_train)
    X_test_pca = pca_obj_this.transform(X_test)
    return (X_train_pca, y_train, X_test_pca, y_test), pca_obj_this

datasets_without_val, pca_obj = load_withoutval_set()

In [5]:
def load_withval_set():
    datasets_with_val = load_split_dataset('MkA_Shape', 'all', True, 553)
    X_train, y_train, X_test, y_test, X_val, y_val = datasets_with_val
    
    X_train = X_train.reshape(X_train.shape[0], -1)
    X_test = X_test.reshape(X_test.shape[0], -1)
    X_val = X_val.reshape(X_val.shape[0], -1)
    
    X_train_pca = pca_obj.transform(X_train)
    X_test_pca = pca_obj.transform(X_test)
    X_val_pca = pca_obj.transform(X_val)
    
    assert np.array_equal(datasets_without_val[2], X_test_pca)
    assert np.array_equal(datasets_without_val[3], y_test)
    
    return (X_train_pca, y_train, X_test_pca, y_test, X_val_pca, y_val)

datasets_with_val = load_withval_set()

In [6]:
# create a model.
def generate_one_model(act_fn='softplus'):
    arch_config = generate_one_config([], {'factored': False, 'dropout': None},
                                     act_fn, False)
    init_config = legacy_generator()
    model_ = CNN(arch_config, init_config, mean_response=datasets_with_val[1].mean(axis=0),
                seed=0)
    model_.cuda()
    return model_


In [7]:
def generate_opt_config(l1_glmnet):
    loss_config = 'mse'
    optimizer_config = cnn_opt.generate_one_optimizer_config('adam', lr=0.001)
    # time 2 because mse doesn't have 1/2 term in it.
    fc_config = cnn_opt.generate_one_layer_opt_config(l1=2*l1_glmnet, l2=0.0, l1_bias=0.0, l2_bias=0.0)
    conv_config = []
    return cnn_opt.generate_one_opt_config(conv_config, fc_config, loss_config,optimizer_config)

In [8]:
# load matlab softplus data.
def load_back_matlab_data():
    with open(os.path.join(dir_dictionary['analyses'], 'glm_R_vs_lassoglm_MATLAB_real_data_softplus.pkl'),
         'rb') as f_debug:
        matlab_stuff = pickle.load(f_debug)
    return matlab_stuff


In [9]:
def softplus(x):
    return np.log(np.exp(x) + 1)

In [10]:
def get_ref_performance():
    matlab_data = load_back_matlab_data()
    
    lambda_idx_mindev = matlab_data['fit_matlab']['FitInfo']['IndexMinDeviance'][0,0][0,0] - 1
    lambda_idx_1se = matlab_data['fit_matlab']['FitInfo']['Index1SE'][0,0][0,0] - 1
    print(lambda_idx_mindev, lambda_idx_1se)
    lambda_mindev = matlab_data['fit_matlab']['FitInfo']['LambdaMinDeviance'][0,0][0,0]
    lambda_1se = matlab_data['fit_matlab']['FitInfo']['Lambda1SE'][0,0][0,0]
    lambda_mindev_debug = matlab_data['fit_matlab']['FitInfo']['Lambda'][0,0].ravel()[lambda_idx_mindev]
    lambda_1se_debug = matlab_data['fit_matlab']['FitInfo']['Lambda'][0,0].ravel()[lambda_idx_1se]
    assert lambda_mindev == lambda_mindev_debug
    assert lambda_1se == lambda_1se_debug
    # print(matlab_data['fit_matlab']['FitInfo']['Intercept'][0,0].shape)
    coeff_mindev, bias_mindev =  (matlab_data['fit_matlab']['B'][:, lambda_idx_mindev],
                                  matlab_data['fit_matlab']['FitInfo']['Intercept'][0,0].ravel()[lambda_idx_mindev])
    coeff_1se, bias_1se =  (matlab_data['fit_matlab']['B'][:, lambda_idx_1se],
                                  matlab_data['fit_matlab']['FitInfo']['Intercept'][0,0].ravel()[lambda_idx_1se])
    
    
    X_train, y_train, X_test, y_test = datasets_without_val
    
    yhat_mindev = softplus(X_test @ coeff_mindev + bias_mindev)
    yhat_1se = softplus(X_test @ coeff_1se + bias_1se)
    assert yhat_mindev.shape == yhat_1se.shape == (y_test.size,)
    
    
    
    result_dict = OrderedDict()
    result_dict['mindev'] = {
        'lambda': lambda_mindev,
        'yhat': yhat_mindev,
        'corr': pearsonr(yhat_mindev, y_test.ravel())[0]
    }
    result_dict['1se'] = {
        'lambda': lambda_1se,
        'yhat': yhat_1se,
        'corr': pearsonr(yhat_1se, y_test.ravel())[0]
    }
    return result_dict


In [11]:
ref_performance_dict = get_ref_performance()

63 99


In [12]:
ref_performance_dict

OrderedDict([('mindev',
              {'corr': 0.17749115977263863,
               'lambda': 9.7354627257237135e-05,
               'yhat': array([ 0.20030271,  0.20784372,  0.23043094, ...,  0.18800964,
                       0.17269229,  0.18015046])}),
             ('1se',
              {'corr': 0.074447123430914364,
               'lambda': 0.0027726947038680919,
               'yhat': array([ 0.19457534,  0.19457534,  0.19457534, ...,  0.19457534,
                       0.19457534,  0.19457534])})])

In [13]:
def train_model(l1_glmnet, act_fn):
    model = generate_one_model(act_fn)
    # it must be float. otherwise, bad things can happen in PyTorch.
    # it will say that np.ndarray cannot be backwarded.
    opt_config = generate_opt_config(float(l1_glmnet))
    print(opt_config)
    print(model.fc.fc)
    y_test_hat, new_cc = train_one_case(model, datasets_with_val,
                                        opt_config, seed=0, show_every=10000)
    print(new_cc)

In [14]:
train_model(ref_performance_dict['1se']['lambda'], 'softplus')
train_model(ref_performance_dict['1se']['lambda'], 'relu')
train_model(ref_performance_dict['mindev']['lambda'], 'softplus')
train_model(ref_performance_dict['mindev']['lambda'], 'relu')

{'conv': [], 'fc': {'l1': 0.005545389407736184, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1600
========end phase 1/1==========
0.110621
{'conv': [], 'fc': {'l1': 0.005545389407736184, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.130645
{'conv': [], 'fc': {'l1': 0.00019470925451447427, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.176126
{'conv': [], 'fc': {'l1': 0.0001

In [15]:
train_model(ref_performance_dict['1se']['lambda']*4, 'softplus')
train_model(ref_performance_dict['1se']['lambda']*4, 'relu')
train_model(ref_performance_dict['mindev']['lambda']*4, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']*4, 'relu')

{'conv': [], 'fc': {'l1': 0.022181557630944735, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1200
========end phase 1/1==========
0.105871
{'conv': [], 'fc': {'l1': 0.022181557630944735, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.108718
{'conv': [], 'fc': {'l1': 0.0007788370180578971, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1600
========end phase 1/1==========
0.147331
{'conv': [], 'fc': {'l1': 0.00077

In [16]:
train_model(ref_performance_dict['1se']['lambda']*2, 'softplus')
train_model(ref_performance_dict['1se']['lambda']*2, 'relu')
train_model(ref_performance_dict['mindev']['lambda']*2, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']*2, 'relu')

{'conv': [], 'fc': {'l1': 0.011090778815472368, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.105604
{'conv': [], 'fc': {'l1': 0.011090778815472368, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 2600
========end phase 1/1==========
0.156376
{'conv': [], 'fc': {'l1': 0.00038941850902894854, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 2800
========end phase 1/1==========
0.166765
{'conv': [], 'fc': {'l1': 0.0003

In [17]:
train_model(ref_performance_dict['1se']['lambda']/2, 'softplus')
train_model(ref_performance_dict['1se']['lambda']/2, 'relu')
train_model(ref_performance_dict['mindev']['lambda']/2, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']/2, 'relu')

{'conv': [], 'fc': {'l1': 0.002772694703868092, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1900
========end phase 1/1==========
0.132546
{'conv': [], 'fc': {'l1': 0.002772694703868092, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1800
========end phase 1/1==========
0.138934
{'conv': [], 'fc': {'l1': 9.735462725723714e-05, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.182021
{'conv': [], 'fc': {'l1': 9.73546

In [18]:
train_model(ref_performance_dict['1se']['lambda']/4, 'softplus')
train_model(ref_performance_dict['1se']['lambda']/4, 'relu')
train_model(ref_performance_dict['mindev']['lambda']/4, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']/4, 'relu')

{'conv': [], 'fc': {'l1': 0.001386347351934046, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1900
========end phase 1/1==========
0.14427
{'conv': [], 'fc': {'l1': 0.001386347351934046, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.141563
{'conv': [], 'fc': {'l1': 4.867731362861857e-05, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.186181
{'conv': [], 'fc': {'l1': 4.867731

In [19]:
train_model(ref_performance_dict['1se']['lambda']/8, 'softplus')
train_model(ref_performance_dict['1se']['lambda']/8, 'relu')
train_model(ref_performance_dict['mindev']['lambda']/8, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']/8, 'relu')

{'conv': [], 'fc': {'l1': 0.000693173675967023, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 2800
========end phase 1/1==========
0.160669
{'conv': [], 'fc': {'l1': 0.000693173675967023, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.151142
{'conv': [], 'fc': {'l1': 2.4338656814309284e-05, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.186877
{'conv': [], 'fc': {'l1': 2.4338

In [20]:
train_model(ref_performance_dict['1se']['lambda']/12, 'softplus')
train_model(ref_performance_dict['1se']['lambda']/12, 'relu')
train_model(ref_performance_dict['mindev']['lambda']/12, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']/12, 'relu')

{'conv': [], 'fc': {'l1': 0.0004621157839780153, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 2800
========end phase 1/1==========
0.164968
{'conv': [], 'fc': {'l1': 0.0004621157839780153, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.157366
{'conv': [], 'fc': {'l1': 1.6225771209539523e-05, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.187064
{'conv': [], 'fc': {'l1': 1.62

In [21]:
train_model(ref_performance_dict['1se']['lambda']/16, 'softplus')
train_model(ref_performance_dict['1se']['lambda']/16, 'relu')
train_model(ref_performance_dict['mindev']['lambda']/16, 'softplus')
train_model(ref_performance_dict['mindev']['lambda']/16, 'relu')

{'conv': [], 'fc': {'l1': 0.0003465868379835115, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 2800
========end phase 1/1==========
0.168124
{'conv': [], 'fc': {'l1': 0.0003465868379835115, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1500
========end phase 1/1==========
0.159291
{'conv': [], 'fc': {'l1': 1.2169328407154642e-05, 'l2': 0.0, 'l1_bias': 0.0, 'l2_bias': 0.0}, 'loss': 'mse', 'optimizer': {'optimizer_type': 'adam', 'lr': 0.001}}
Linear(in_features=400, out_features=1, bias=True)
========starting phase 1/1==========
early stopping after epoch 1100
========end phase 1/1==========
0.186838
{'conv': [], 'fc': {'l1': 1.21